## WORDNXT :: Next Word Predictor

#### NOTE: This piece of code is first part of the project that is RNN model. This model has been trained on LSTN, a text specific model. There will be more changes in upcoming developments of this code and project as a whole.

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os
import string

In [ ]:
#pip install tensorflow-gpu --user

In [ ]:
#sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

In [ ]:
#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

In [2]:
file = open("WikiQA-train.txt", "r", encoding = "utf8")
lines = []

for i in file:
    lines.append(i)
    
data = ""

for i in lines:
    data = ' '. join(lines)
    
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '')

translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
new_data = data.translate(translator)

z = []

for i in data.split():
    if i not in z:
        z.append(i)
        
data = ' '.join(z)

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

# saving the tokenizer for predict function.
pickle.dump(tokenizer, open('tokenizer1.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]

vocab_size = len(tokenizer.word_index) + 1

sequences = []

for i in range(1, len(sequence_data)):
    words = sequence_data[i-1:i+1]
    sequences.append(words)
    
sequences = np.array(sequences)

X = []
y = []

for i in sequences:
    X.append(i[0])
    y.append(i[1])
    
X = np.array(X)
y = np.array(y)

y = to_categorical(y, num_classes=vocab_size)

In [4]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

In [5]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard

checkpoint = ModelCheckpoint("nextword1.h5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto')

reduce = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose = 1)

logdir='logsnextword1'
tensorboard_Visualization = TensorBoard(log_dir=logdir)

In [6]:
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.001))
model.fit(X, y, epochs=10, batch_size=64, callbacks=[checkpoint, reduce, tensorboard_Visualization])


Epoch 1/10
700/700 [==============================] - 161s 226ms/step - loss: 10.3572

Epoch 00001: loss improved from inf to 10.37270, saving model to nextword1.h5
Epoch 2/10
700/700 [==============================] - 155s 221ms/step - loss: 10.1006

Epoch 00002: loss improved from 10.37270 to 10.12188, saving model to nextword1.h5
Epoch 3/10
700/700 [==============================] - 152s 218ms/step - loss: 9.7368

Epoch 00003: loss improved from 10.12188 to 9.74586, saving model to nextword1.h5
Epoch 4/10
700/700 [==============================] - 153s 218ms/step - loss: 9.0289

Epoch 00004: loss improved from 9.74586 to 9.02385, saving model to nextword1.h5
Epoch 5/10
700/700 [==============================] - 151s 216ms/step - loss: 8.2223

Epoch 00005: loss improved from 9.02385 to 8.25955, saving model to nextword1.h5
Epoch 6/10
700/700 [==============================] - 152s 218ms/step - loss: 7.4405

Epoch 00006: loss improved from 8.25955 to 7.49924, saving model to nextword1